In [5]:
import json
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

# load the JSON data
num_classes = 6
with open('training.json', 'r') as f:
    data_dict = json.load(f)

obj = {
    "Hand Close": 0,
    "Thumb": 1,
    "Index": 2,
    "Middle": 3,
    "Ring": 4,
    "Pinky": 5
}
    
# extract the x and y data from the JSON dictionary
xs_train = [[int(x) for x in data['xs'].values()] for data in data_dict['data']]
ys_train = [obj[data['ys']['label']] for data in data_dict['data']]
ys_train = tf.keras.utils.to_categorical(ys_train, num_classes)

xs_train = np.array(xs_train)
ys_train = np.array(ys_train)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(xs_train, ys_train, test_size = 0.25, random_state = 0) 

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X = scaler.transform(xs_train)

# define the number of folds
k = 5

# create a KFold object
kf = KFold(n_splits=k, shuffle=True)

# iterate over the folds
for fold, (train_index, test_index) in enumerate(kf.split(X)):
    # get the training and test data for this fold
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = ys_train[train_index], ys_train[test_index]

    # normalize the data
    x_train = (x_train - x_train.min()) / (x_train.max() - x_train.min())
    x_test = (x_test - x_train.min()) / (x_train.max() - x_train.min())

    # create the model
    model = Sequential([
        tf.keras.layers.Dense(58, activation='relu', input_shape=(999,)),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(58, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(58, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(58, activation='relu'),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(58, activation='relu'),
        #tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(58, activation='relu'),
        #tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(num_classes, activation='softmax'),
    ])

    # compile the model
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    # train the model on the training data
    model.fit(x_train, y_train, epochs=1000, batch_size=1000, verbose=0)

    # evaluate the model on the test data
    loss, accuracy = model.evaluate(x_test, y_test, batch_size=32, verbose=0)

    # print the test accuracy for this fold
    print(f'Fold {fold+1} Test accuracy: {accuracy:.4f}')


Fold 1 Test accuracy: 0.2381
Fold 2 Test accuracy: 0.0476
Fold 3 Test accuracy: 0.0500
Fold 4 Test accuracy: 0.0500
Fold 5 Test accuracy: 0.0500
